In [ ]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051

In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

update_line_webhook(webhook_url)

Ngrok URL: https://whistleable-lupe-perfunctory.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://whistleable-lupe-perfunctory.ngrok-free.dev


True

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch


client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)



In [ ]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology），簡稱明新科大，是一所位於臺灣新竹縣新豐鄉的私立科技大學。

**歷史沿革**
學校創立於1966年，最初以「明新工業專科學校」立案，設有機械、土木、工業管理等科別。 歷經數次改制，於1997年改為「明新技術學院」，並最終在2002年獲教育部核准升格為「明新科技大學」。2018年，更名為「明新學校財團法人明新科技大學」。

**辦學理念與特色**
明新科技大學秉持「堅毅、求新、創造」的校訓精神，以培育具備專業技能、團隊精神、人文素養與宏觀服務的科技人才為目標。學校的願景是「深耕在地、放眼國際」，並致力於成為一所「一流產業大學」。

學校辦學強調：
*   **產學連結**：緊鄰新竹科學園區與新竹工業區，擁有豐富的產學資源，透過契合式人才課程，使畢業生能「立即就業」。
*   **跨域整合**：發展多元學習、全球視野、永續經營與技術創新等四大育才特色，鼓勵學生跨域學習。
*   **產業導向**：積極配合產業趨勢，整合資源，例如成立半導體學院，並導入生成式AI等技術於教學與實習場域，發展智慧零售、智慧金融等實驗室。
*   **國際化**：推動跨國人才培育，與菲律賓、澳洲西雪梨大學及越南胡志明市經濟大學等簽訂雙聯學位合作。

**學術單位**
明新科技大學設有半導體學院、工程學院、管理學院、民生學院、人文與設計學院、共同教育學院等六個學院。旗下涵蓋多個學系、碩士班及學位學程，其中半導體科技博士學位學程是學校首個博士班。

**社會肯定**
明新科大的辦學績效受到肯定，在2022年《遠見》雜誌的「企業最愛公私立技職科大調查」中，獲得兩項企業最愛，起薪排名私校第一，甚至優於部分國立科大。根據1111人力銀行統計，明新科大在半導體產業最愛聘用的畢業生中名列前茅，與頂尖大學並列前五。


In [ ]:
result2 = stateful_query("校長是誰？")
print(result2)

我需要更多資訊才能回答您的問題。請問您是指哪所學校或機構的校長？


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']

    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit


BODY:  {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[{"type":"message","message":{"type":"text","id":"595680732782264440","quoteToken":"bLK36MkTDFqKjdeQyr9nF-Bn_zy1JxA9YwkbdGzjZ28x_pT75CDqGxj_8L-QmByz4BcvmPVXf7Q2BUiJMsDvnwWNOmS4Uqvb6xbzlfXMiVKqs944zGg3ZXV2CQk8Txr323WfpaXs2pKd4UyyJ3720A","markAsReadToken":"OZJwFCLFsVPZXA4HYzQWnjweg2tjTCI6h8BWz2VGMWO9CYt96xARVT4Xp-V59oHoKKGMizRAcSByH9dys9vf4CR6og4pkePh-kXUBL0-zQskW9dFlDH_wHntbqMiJ0FYUxaXl5qbM55LhhQh3RBdhL_HYuWt1cNAPO2TCgQszkKXxr6kzb_8Fq3LK-hEhHjz_4HsHsDgB3PP6CmULKxwKA","text":"AI search about the most itellegence ai"},"webhookEventId":"01KEF273W4ZS0860E8FG4DE8A3","deliveryContext":{"isRedelivery":false},"timestamp":1767884688775,"source":{"type":"user","userId":"U9bce42694348bd9a01e5e0f0f4f36da9"},"replyToken":"13f795b07b69471eb7b1ee7b674a9ba8","mode":"active"}]}


ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1799486110.py", line 39, in callback
    handler.handle(body, signature)
  File "/usr/local/lib/python3.12/di

BODY:  {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[]}
BODY:  {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[{"type":"message","message":{"type":"text","id":"595680858611384508","quoteToken":"FXgkJLbbQpJbjMGJvUHCjYbF-ksYt3u95zBGp0tiahXdAJWqI3g3F1Fr1C4To-xZAzbDkYlNwccJW9mpgxHuPM4F30e_0e3y_o3AvXOEE0pVu9GjJ5mFLc7rU076x12esxYWaXdEZaGhE4jJYPQUXg","markAsReadToken":"CF94wwK2p6PaAJnn_UpAU58S8fYcC0ZVDr0Kipg-JBrvjzmicJdoh86lLgcCL2CLIkB9AqZnFMeLBJ5Kyw0TIMeil10vj3TzW_xsjWd-NpbzmruAiG7Pzvhw7Na0AYDjxbxFsAHmms1SSKh8b--uTrzqut8vY8knVV8m0c3nUPnDedhtz9WdofWb1dTfur-FVSkD6MpL3_K0o_vaGXmTCw","text":"AI what language you using"},"webhookEventId":"01KEF29D3Z4EM58178YR7B795W","deliveryContext":{"isRedelivery":false},"timestamp":1767884763778,"source":{"type":"user","userId":"U9bce42694348bd9a01e5e0f0f4f36da9"},"replyToken":"5e1bd5597e0549c88c1073a7fcd33dc6","mode":"active"}]}


...

INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 15:06:27] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[{"type":"message","message":{"type":"text","id":"595680962998960614","quoteToken":"DdwtUVvoTNky3yfTgrTD2oMuAeW9ox0rV3dJ9aSdq6pND7pgMCubPpNAtZ0rrAK9q5wPK-19iKA2fNx4_dbmjs-nG4YAeDx0Lf19lnXAdEJZx4sMtsPBx2zyjq2wFZbu6Oa6pHkGfGD6z6tWJ4DmOQ","markAsReadToken":"A-KPWwR57YHheWXVB-zD32S_wCUymlqao5TXBROt1_DCzqmJNu0Xr2IgHDUbI-KpSjkUwWWMvCj3vEujBa7sPj4MCA1UfJySHDZ3JzCyiFNCHtc4T09OP-miuCYv1Mot4qPVMMt3zUMhFibTpby39vy7XsdK1J5TtLUVr1XpbQdo9LR942KWjbMqqTqEjlq1i9F8KnR_DxwEqktsMmZLmg","text":"can you translate to english"},"webhookEventId":"01KEF2BA2BH96BCB159WAEBRWN","deliveryContext":{"isRedelivery":false},"timestamp":1767884826191,"source":{"type":"user","userId":"U9bce42694348bd9a01e5e0f0f4f36da9"},"replyToken":"bf509b7c40434725bfaa09d5830e94e2","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 15:07:07] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[{"type":"message","message":{"type":"text","id":"595680986000785783","quoteToken":"eI1Bl64zx0_zWXwxEEIs-OgWz-pOPbLBfzFSxeMlXRr7P8hSJ3BBNwA95eJ5OdnlYLFtpn1oNLda-85m8m__wgEga8zBTxucj1Ev_xhdpQBF9wDOqdXo8W-zTR46WMm76mCpBrDLq75OcEj7xgfi2A","markAsReadToken":"HajEZCINsC0n19wvyjmMlZPhf515nc6bvGuR-fWDnJG9xulnfuqRUVmXeUL3UoFxOfKTr-26qHH3miFjXkrpP79luj74_WQtenGmpWhRGR8MzSNJtNf8KusMBpwm_E9jkc5pJPMSqJFl5tnyUYN8C8a4VNAyW_QZIk4i2S6l6Klc59UARNcQH8px8aDiraqWMRy6y_dUjVLs-R1aSUabmA","text":"AI can you translate to english"},"webhookEventId":"01KEF2BQ1XWMXJ2QX3NTCD8HGN","deliveryContext":{"isRedelivery":false},"timestamp":1767884839713,"source":{"type":"user","userId":"U9bce42694348bd9a01e5e0f0f4f36da9"},"replyToken":"1ced15c37f074b2291920ea9935fbb81","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 15:07:22] "POST / HTTP/1.1" 200 -
